### Linear regression

## Import vers sql

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import ElasticNet,  Ridge, Lasso
from sklearn.preprocessing import PowerTransformer
import joblib

In [2]:
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv
from sqlalchemy import create_engine
import os

In [3]:
load_dotenv()

True

In [4]:
user = os.getenv("PGUSER")
password = os.getenv("PGPASSWORD")
host = os.getenv("PGHOST")
port = os.getenv("PGPORT")
database = os.getenv("PGDATABASE")

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

In [5]:
def import_table_to_dataframe(table_name):
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, engine)
    return df

data_clean = import_table_to_dataframe("immo")
print(data_clean.head())

       price    city_name  salon  nb_rooms  nb_baths  surface_area  Ascenseur  \
0  2000000.0   Casablanca    1.0       2.0       2.0         168.0          1   
1  1195000.0   Casablanca    1.0       2.0       2.0          98.0          1   
2  1350000.0  Dar Bouazza    1.0       2.0       2.0          81.0          1   
3   900000.0   Casablanca    1.0       1.0       1.0          56.0          1   
4  3100000.0        Rabat    2.0       3.0       2.0         200.0          1   

   Balcon  Chauffage  Climatisation  ...  Meublé  Parking  Sécurité  Terrasse  \
0       1          0              0  ...       0        1         0         1   
1       1          1              1  ...       0        1         0         0   
2       1          1              1  ...       0        1         1         1   
3       0          1              1  ...       1        1         1         1   
4       1          0              0  ...       0        1         1         0   

   ville_encoded  ville_nb

## Sélection des variables explicatives

In [6]:
colonnes_selectionnees=['surface_area', 'ville_encoded', 'equipement_tot', 'nb_tot', 'Ascenseur']

## Séparation des données

In [7]:
X=data_clean[colonnes_selectionnees]
y=data_clean['price']

X_train, X_test, y_train, y_test=train_test_split(X, y,test_size=0.20,random_state=42 )
y_train=y_train.ravel()
X_train.shape, y_train.shape

C:\Users\lenovo\AppData\Local\Temp\ipykernel_716\3320900413.py:5: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_train=y_train.ravel()


((11644, 5), (11644,))

### Régression Linéaire

Pour la regression linéaire on choisit le GridSearchCV car il y a peu d'hyperparamètres et l espace de recherche est petit.

In [ ]:
Model_1 = Pipeline([
    ('scaler', StandardScaler()),
    ('model', TransformedTargetRegressor(
        regressor=LinearRegression(), 
        func=np.log1p,                
        inverse_func=np.expm1         
    ))
])

In [ ]:
param_grid = {
    'model__regressor__fit_intercept': [True, False]  
}

In [10]:
grid_search = GridSearchCV(Model_1, param_grid, scoring='r2', cv=5)
grid_search.fit(X_train, y_train)

,estimator,Pipeline(step...gression()))])
,param_grid,"{'model__regressor__fit_intercept': [True, False]}"
,scoring,'r2'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,copy,True


In [11]:
best_model_RL = grid_search.best_estimator_
print(f"Meilleur paramètre : {grid_search.best_params_}")

Meilleur paramètre : {'model__regressor__fit_intercept': True}


In [12]:
cv_results = cross_validate(best_model_RL, X_train, y_train, cv=3, scoring={'neg_mse': 'neg_mean_squared_error', 
                                                                            'neg_mae': 'neg_mean_absolute_error', 
                                                                            'r2': 'r2'})

mse_scores = -cv_results['test_neg_mse']
rmse_scores = np.sqrt(mse_scores)
mae_scores = -cv_results['test_neg_mae']
r2_scores = cv_results['test_r2']

print(f"MSE moyen CV : {mse_scores.mean():.3f} ± {mse_scores.std():.3f}")
print(f"RMSE moyen CV : {rmse_scores.mean():.3f} ± {rmse_scores.std():.3f}")
print(f"MAE moyen CV  : {mae_scores.mean():.3f} ± {mae_scores.std():.3f}")
print(f"R² moyen CV   : {r2_scores.mean():.3f} ± {r2_scores.std():.3f}")

MSE moyen CV : 397145697347.389 ± 13470741958.331
RMSE moyen CV : 630104.023 ± 10705.952
MAE moyen CV  : 364371.558 ± 5275.360
R² moyen CV   : 0.370 ± 0.028


In [13]:
# Évaluation finale sur le jeu test
best_model_RL.fit(X_train, y_train)
y_pred_test = best_model_RL.predict(X_test)

mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print("\nPerformance sur jeu test :")
print(f"MSE  : {mse_test:.3f}")
print(f"RMSE : {rmse_test:.3f}")
print(f"MAE  : {mae_test:.3f}")
print(f"R²   : {r2_test:.3f}")


Performance sur jeu test :
MSE  : 330143407418.306
RMSE : 574581.071
MAE  : 347297.984
R²   : 0.407
